
# MULTICAST

Apartado 1

Este cliente está escuchando constantemente en el grupo de multicast hasta que reciba un mensaje de Adios, en ese momento abandonará el grupo y cerrará la conexión

In [1]:
import socket
import struct

MCAST_GRP = '224.1.1.1'
MCAST_PORT = 5007

sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM, socket.IPPROTO_UDP)
sock.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)

#Usar '' para esuchar y escribir en el socket
#Usar MCAST_GRP para escuchar solo en el socket
sock.bind(('', MCAST_PORT))
sock.sendto(str.encode("Hello this is user 2"), (MCAST_GRP, MCAST_PORT))

mreq = struct.pack("4sl", socket.inet_aton(MCAST_GRP), socket.INADDR_ANY)
sock.setsockopt(socket.IPPROTO_IP, socket.IP_ADD_MEMBERSHIP, mreq)

#Establecer el ttl
ttl = struct.pack('b',1)
sock.setsockopt(socket.IPPROTO_IP, socket.IP_MULTICAST_TTL, ttl)

msg = ""
while str(msg) != "Adios":
    msg = sock.recv(1024).decode()
    print("RX: ", msg)
    
print("Cliente abandona el grupo")                       
sock.close() 

RX:  hola
RX:  Adios
Cliente abandona el grupo


Apartado 2

Este cliente es una copia del cliente del otro notebook que participa en el grupo

In [6]:
import socket
import struct
import pickle
import threading

MCAST_GRP = '224.1.1.1'
MCAST_PORT = 5007

# identificador del cliente
user_id = 2

sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM, socket.IPPROTO_UDP)
sock.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)

#Usar '' para esuchar y escribir en el socket
#Usar MCAST_GRP para escuchar solo en el socket
sock.bind(('', MCAST_PORT))
mensaje_bienvenida = 'Hello this is user 2'
bytes_bienvenida = pickle.dumps([mensaje_bienvenida, user_id])
sock.sendto(bytes_bienvenida, (MCAST_GRP, MCAST_PORT))

mreq = struct.pack("4sl", socket.inet_aton(MCAST_GRP), socket.INADDR_ANY)
sock.setsockopt(socket.IPPROTO_IP, socket.IP_ADD_MEMBERSHIP, mreq)

#Establecer el ttl
ttl = struct.pack('b', 1)
sock.setsockopt(socket.IPPROTO_IP, socket.IP_MULTICAST_TTL, ttl)

def time_function(name):
    global stop
    while stop == False:
        msg = sock.recv(1024)
        mensaje = pickle.loads(msg)
        print("Thread User " + str(mensaje[1]) + ": " + str(mensaje[0]))
    
    print("Cliente abandona el grupo")
    sock.close()
    
stop = False
reader = threading.Thread(target=time_function, args=(user_id,))
reader.start()

while stop == False:
    mensaje_tx = input('Introduzca el mensaje a enviar: ')
    bytes_tx = pickle.dumps([mensaje_tx, user_id])
    sock.sendto(bytes_tx, (MCAST_GRP, MCAST_PORT))
    if mensaje_tx == "quit":
        stop = True
        break
                          
sock.close()

Introduzca el mensaje a enviar: holaa
Thread User 2: holaa
Introduzca el mensaje a enviar: quit
Thread User 2: quit


Exception in thread Thread-10:
Traceback (most recent call last):
  File "C:\Users\javie\anaconda3\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "C:\Users\javie\anaconda3\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-6-295baae872ba>", line 32, in time_function
OSError: [WinError 10038] Se intentó realizar una operación en un elemento que no es un socket
